# Compilation du modèle avec ONNX

## 1 - Préparation et compilation du modèle

In [32]:
import torch
import torchvision
from collections import OrderedDict

In [33]:
# Créé à l'aide de ChatGPT
import torch

class ReshapeToBatchChannelFirst(torch.nn.Module):
    def __init__(self):
        super(ReshapeToBatchChannelFirst, self).__init__()

    def forward(self, x):
        # Ensure the input is of shape (224, 224, 3)
        #assert x.dim() == 3 and x.shape[-1] == 3, "Input must be (224, 224, 3)"
        
        # Permute dimensions from (H, W, C) to (C, H, W)
        x = x.permute(2, 0, 1)
        
        # Add a batch dimension at the beginning: (1, C, H, W)
        x = x.unsqueeze(0)
        return x

# Example usage
x = torch.rand(224, 224, 3)  # Example input
layer = ReshapeToBatchChannelFirst()
output = layer(x)
print(output.shape)  # Expected: torch.Size([1, 3, 224, 224])


torch.Size([1, 3, 224, 224])


In [35]:
# Créé à l'aide de ChatGPT
import torch
import torch.nn as nn

class FixedNormLayer(torch.nn.Module):
    def __init__(self, scale: torch.Tensor, mean: torch.Tensor, std: torch.Tensor):
        """
        Args:
            mean (torch.Tensor): Precomputed mean for normalization.
            std (torch.Tensor): Precomputed standard deviation for normalization.
        """
        super(FixedNormLayer, self).__init__()
        self.register_buffer("mean", mean[:, None, None])
        self.register_buffer("std", std[:, None, None])
        self.register_buffer("scale", scale)

    def forward(self, x):
        return (self.scale * x - self.mean) / self.std

# Example usage
mean = torch.tensor([0.5, 0.5, 0.5])  # Example mean for 3 channels
std = torch.tensor([0.2, 0.2, 0.2])   # Example std for 3 channels
scale = torch.tensor([1 / 256])
layer = FixedNormLayer(scale, mean, std)

# Test with a sample input
x = torch.rand(1, 3, 1, 1)  # Example input
output = layer(x)
print(output)

tensor([[[[-2.4883]],

         [[-2.4994]],

         [[-2.4831]]]])


In [38]:
# Créé à l'aide de ChatGPT
class InferenceModel(nn.Module):
    def __init__(self, model, scale, mean, std):
        super(InferenceModel, self).__init__()
        self.preprocess = nn.Sequential(
            OrderedDict(
                [
                    ("reshape", ReshapeToBatchChannelFirst()),
                    ("normalize", FixedNormLayer(scale, mean, std)),
                ]
            )
        )
        self.model = model  # The main model
        self.postprocess = torch.nn.Softmax(1)

    def forward(self, x):
        x = self.preprocess(x)  # Apply reshaping and normalization
        x = self.model(x)  # Pass to the main model
        return self.postprocess(x)

Modification du modèle pour avoir le bon nombre de sortie dans la dernière couche et
pour calculer le softmax sur les sorties du modèle pour avoir directement les probabilités.

In [39]:
# Load pretrained ViT
num_labels = 11  # Get number of labels (e.g., 8)

model = torchvision.models.vit_b_16(weights="IMAGENET1K_V1")  # Load a pretrained model
model.heads.head = torch.nn.Linear(model.heads.head.in_features, num_labels)

In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model.load_state_dict(
    torch.load(
        "models/tomato_model_2025_02_28_v2.pt",
        map_location=device,
        weights_only=True,
    )
)

<All keys matched successfully>

In [42]:
# Create inference model
scale = torch.tensor([1 / 256])
mean = torch.tensor([0.485, 0.456, 0.406])
std = torch.tensor([0.229, 0.224, 0.225])
inference_model = InferenceModel(model, scale, mean, std)
inference_model.eval()

test = torch.randn(224, 224, 3)
inference_model(test)

tensor([[0.0214, 0.6052, 0.2213, 0.0180, 0.0034, 0.0022, 0.0020, 0.0813, 0.0186,
         0.0066, 0.0200]], grad_fn=<SoftmaxBackward0>)

Exportation du modèle, en incluant un tenseur aléatoire pour fournir la bonne taille de
tenseur en entrée.

In [43]:
torch_input = torch.randn(224, 224, 3)
#onnx_program = torch.onnx.export(inference_model, torch_input, opset_version=14)
onnx_program = torch.onnx.dynamo_export(inference_model, torch_input)

/home/maxime/.venvs/ai/lib64/python3.11/site-packages/torch/onnx/_internal/_exporter_legacy.py:116: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(
/home/maxime/.venvs/ai/lib64/python3.11/site-packages/torch/onnx/_internal/fx/passes/readability.py:52: UserWarning: Attempted to insert a get_attr Node with no underlying reference in the owning GraphModule! Call GraphModule.add_submodule to add the necessary submodule, GraphModule.add_parameter to add the necessary Parameter, or nn.Module.register_buffer to add the necessary buffer
  new_node = self.module.graph.get_attr(normalized_name)
/home/maxime/.venvs/ai/lib64/python3.11/site-packages/torch/fx/graph.py:1586: UserWarning: Node preprocess_normalize_scale target preprocess/normalize/scale preprocess/normalize/scale of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_at

Applied 37 of general pattern rewrite rules.


Ces warnings ne sont probablement pas grave, selon cette
[source](https://github.com/pytorch/pytorch/issues/144331).

In [44]:
onnx_program.save("models/tomato_model_2025_02_28_v2.onnx")

## 2 - Validation de l'exécution du modèle avec ONNX runtime

In [6]:
import onnxruntime
import PIL.Image
import numpy as np

ort_session = onnxruntime.InferenceSession(
    "models/tomato_model_2025_02_28_v2.onnx", providers=["CPUExecutionProvider"]
)

Pipeline fait sans pytorch

In [7]:
def single_image_pipeline(image_path, dtype="float32"):
    # Load image into numpy float array
    image = np.array(
        PIL.Image.open(image_path).convert("RGB").resize((224, 224)), dtype=dtype
    )

    return image

In [8]:
# Exécution du modèle
onnx_input = single_image_pipeline(
    "dataset/tomato/88614302-e6d2-4327-a4fb-a3db9c9ea72e___YLCV_NREC_2861.JPG"
)

onnxruntime_outputs = ort_session.run(None, {"l_x_": onnx_input})
onnxruntime_outputs

[array([[1.4499942e-06, 9.2473765e-06, 2.0949008e-06, 8.7807439e-06,
         1.5942234e-05, 7.3268388e-06, 1.2185769e-06, 9.3832878e-06,
         7.4934546e-06, 1.7394845e-05, 9.9991953e-01]], dtype=float32)]